In [1]:
import tensorflow as tf
import numpy as np
import Region_Proposal_Network as rpn
import ROI_RCNN as rcnn
import extract_feature as vgg_rpn
import os
import time

In [2]:
LEARNING_RATE_BASE = 0.0001
LEARNING_RATE_DECAY = 0.1
MOMENTUM =0.9
TRAIN_NUM = 400
LOOP_TIMES =1

In [3]:
checkpoint_steps = 1000
checkpoint_dir = '/path/to/save_model/fasterRCNN/'

In [4]:
def _modified_smooth_l1(bbox_pred, bbox_targets, bbox_inside_weights,sigma):
    """
        ResultLoss = outside_weights * SmoothL1(inside_weights * (bbox_pred - bbox_targets))
        SmoothL1(x) = 0.5 * (sigma * x)^2,    if |x| < 1 / sigma^2
                      |x| - 0.5 / sigma^2,    otherwise
    """
    sigma2 = sigma * sigma
    inside_mul = tf.multiply(bbox_inside_weights, tf.subtract(bbox_pred, bbox_targets))

    smooth_l1_sign = tf.stop_gradient(tf.cast(tf.less(tf.abs(inside_mul), 1.0 / sigma2), tf.float32))
    smooth_l1_option1 = tf.multiply(tf.multiply(inside_mul, inside_mul), 0.5 * sigma2)
    smooth_l1_option2 = tf.subtract(tf.abs(inside_mul), 0.5 / sigma2)
    smooth_l1_result = tf.add(tf.multiply(smooth_l1_option1, smooth_l1_sign),
                              tf.multiply(smooth_l1_option2, tf.abs(tf.subtract(smooth_l1_sign, 1.0))))
    outside_mul = tf.reduce_sum(smooth_l1_result)
    #outside_mul = tf.multiply(bbox_outside_weights, smooth_l1_result)
    return outside_mul

def _fast_rcnn_loc_loss(pred_loc, gt_loc, gt_label, sigma):
    in_weight0 = tf.zeros([tf.shape(gt_loc)[0],tf.shape(gt_loc)[1]],dtype=tf.int32)
    in_weight1 = tf.ones([tf.shape(gt_loc)[0],tf.shape(gt_loc)[1]],dtype=tf.int32)
    in_weight = tf.where((gt_label > 0),in_weight1,in_weight0)
    loc_loss = _modified_smooth_l1(pred_loc, gt_loc, in_weight, sigma)
    loc_loss /= tf.to_float(tf.shape(tf.where(gt_label>=0))[0])
    return 10*loc_loss

In [5]:
#####重新整理损失函数，一定用tensor计算，输入图像格式换为tf.placeholder
def train():
    #################  img input #################
    img_input = tf.placeholder(tf.float32,shape=[1,None,None,3],name='img_input')
    gt_rpn_label = tf.placeholder(tf.int32,shape=[None],name='gt_rpn_label')
    gt_rpn_loc = tf.placeholder(tf.float32,shape=[None,4],name='gt_rpn_loc')
    sample_roi = tf.placeholder(tf.float32,shape=[None,4],name='sample_roi')
    gt_roi_loc = tf.placeholder(tf.float32,shape=[None,4],name='gt_roi_loc')
    gt_roi_label = tf.placeholder(tf.int32,shape=[None],name='gt_roi_label')
    roi_loc_index = tf.placeholder(tf.int32,shape=[None,4,2],name='roi_loc_index')
    #################  VGG+RPN #################
    feature,rpn_label,rpn_loc = vgg_rpn.extra_feature(img_input)
    rpn_label_softmax = tf.nn.softmax(rpn_label)
    #################  计算RPN损失函数 #################
    rpn_pos_logits = tf.reshape(tf.gather(rpn_label,tf.where(tf.not_equal(gt_rpn_label,-1))),[-1,2])
    rpn_label_toloss = tf.reshape(tf.gather(gt_rpn_label,tf.where(tf.not_equal(gt_rpn_label,-1))),[-1])
    rpn_cls_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=rpn_pos_logits,labels=rpn_label_toloss))
    rpn_rgr_loss = _fast_rcnn_loc_loss(rpn_loc, gt_rpn_loc, gt_rpn_label, sigma=3)###调整到这一步
    rpn_loss = rpn_cls_loss + rpn_rgr_loss

    ###################   RCNN 分类与回归结果   ##################
    fc_cls_21,fc_rgr_84 = rcnn.VGG16RoIHead(sample_roi,feature)
    pre_cls_21 = tf.nn.softmax(fc_cls_21)
    ##################  选出标记label对应的四个loc信息

    pre_loc = tf.gather_nd(fc_rgr_84,roi_loc_index) ###修正
    ###################   RCNN损失计算   ##################
    roi_cls_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=fc_cls_21 , labels=gt_roi_label))
    roi_rgr_loss = _fast_rcnn_loc_loss(pre_loc, gt_roi_loc, gt_roi_label, sigma=1)###
    roi_rgr_loss = roi_rgr_loss;
    roi_loss = roi_cls_loss + roi_rgr_loss 
    loss = rpn_loss + roi_loss
    roi_pos_nums = tf.to_float(tf.shape(tf.where(gt_roi_label>=1))[0])
    
    ###################设置学习率，训练方式
    global_step = tf.Variable(0, trainable=False,name="global_step")
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,30000, LEARNING_RATE_DECAY,staircase=True)
    #train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    train_op = tf.train.MomentumOptimizer(learning_rate,MOMENTUM).minimize(loss,global_step=global_step)

    tf.summary.scalar('rpn_cls_loss', rpn_cls_loss)
    tf.summary.scalar('rpn_rgr_loss', rpn_rgr_loss)
    tf.summary.scalar('roi_cls_loss', roi_cls_loss)
    tf.summary.scalar('roi_rgr_loss', roi_rgr_loss)
    tf.summary.scalar('all_loss', loss)
    tf.summary.scalar('learning_rate', learning_rate)
    merged = tf.summary.merge_all()
    ###################   开始计算   ##################
    saver = tf.train.Saver()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        summary_writer = tf.summary.FileWriter(r"/path/to/board_test",sess.graph)
        tf.global_variables_initializer().run()
        train_path = os.path.join('data',"traindata","img")
        path = os.path.join('data',"traindata")
        train_data = os.listdir(train_path)   
        loss_min = 20;
        for j in range(LOOP_TIMES):
            for i in range(TRAIN_NUM):
                train_name = train_data[i].strip().split('.')[0]
                img,gt_bbox_temp,gt_label_temp,scale =vgg_rpn.get_img_sample(path,train_name)
                img = np.array(img,dtype=float)
                img_mean = img.copy()
                img_mean -= np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
                __b,img_H,img_W,__c = img.shape 
                feat_stride = 16
                F_H = (img_H+feat_stride-1)//feat_stride
                F_W = (img_W+feat_stride-1)//feat_stride
                rpn_label_temp,rpn_loc_tmp = sess.run([rpn_label_softmax,rpn_loc],feed_dict={img_input:img_mean})
                ROI_bbox,anchor = rpn.Region_Proposal_Network(rpn_label_temp,rpn_loc_tmp,img_W,img_H)
                gt_rpn_label_temp,gt_rpn_loc_temp = rpn.AnchorTargetCreator(gt_bbox_temp,anchor,F_W*16,F_H*16)
                ################### e  ROI区域选择，生成标记数据   ##################
                sample_roi_temp, gt_roi_loc_temp, gt_roi_label_temp = rcnn.ProposalTargetCreator(ROI_bbox,
                                                                    gt_bbox_temp,gt_label_temp) ##用于反向传播
                batch_roi = gt_roi_label_temp.shape[0]
                if batch_roi!=64:
                    print("i is",i,"batch_roi is",batch_roi)
                    continue;
                label_index = np.zeros([batch_roi,4,2])
                temp = np.array([[i] for i in range(batch_roi)],dtype=int)
                label_index[:,:,0] = temp 
                label_index[:,:,1] = [[gt_roi_label_temp[i]*4,gt_roi_label_temp[i]*4+1,gt_roi_label_temp[i]*4+2,
                                       gt_roi_label_temp[i]*4+3] for i in range(batch_roi)]
                label_index = np.array(label_index,dtype=int)
                feed_dict_roi = {gt_rpn_label:gt_rpn_label_temp, gt_rpn_loc:gt_rpn_loc_temp,
                                sample_roi:sample_roi_temp, gt_roi_loc:gt_roi_loc_temp, img_input:img_mean,
                                gt_roi_label:gt_roi_label_temp,roi_loc_index:label_index}
                summary,__train_,loss_temp, rpnclsloss, rpnrgrloss, roiclsloss,roirgrloss,\
                            label_output,loc_output,pre_loc_temp,roi_pos_nums_tmp\
                = sess.run([merged,  train_op,   loss,rpn_cls_loss, rpn_rgr_loss, roi_cls_loss, roi_rgr_loss,\
                            pre_cls_21,fc_rgr_84,pre_loc,roi_pos_nums],feed_dict=feed_dict_roi)
                if (i ==LOOP_TIMES-1)&(loss_temp<loss_min):
                    saver.save(sess, checkpoint_dir + 'model.ckpt', global_step=i+1)
                if (i+1)%(TRAIN_NUM/10)==0:
                    print(j,i,"rpnclsloss is ",rpnclsloss,"rpnrgrloss is ",rpnrgrloss,"roiclsloss is ",
                          roiclsloss,"roirgrloss is ",roirgrloss,"roi_pos_nums_tmp",roi_pos_nums_tmp)
                summary_writer.add_summary(summary, i)
    
    summary_writer.close()
    ##tensorboard --logdir  /path/to/board_test 
    return rpn_label_temp,rpn_loc_tmp,label_output,loc_output\
        ,sample_roi_temp,gt_roi_loc_temp, gt_roi_label_temp,pre_loc_temp,gt_rpn_label_temp\
        ,gt_rpn_loc_temp,anchor,gt_bbox_temp,rpn_loc_tmp,rpn_label_temp,label_index

In [6]:
time_start=time.time()
rpn_label_temp,rpn_loc_tmp,label_output,loc_output,sample_roi,gt_roi_loc, gt_roi_label,roi_pre_loc,gt_rpn_label,\
    gt_rpn_loc,anchor,gt_bbox,pre_rpn_loc,pre_rpn_label ,label_index= train()
time_end=time.time()

D:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 39 rpnclsloss is  0.694921 rpnrgrloss is  4.30846 roiclsloss is  0.774356 roirgrloss is  0.0581591 roi_pos_nums_tmp 4.0
error roi img batch is not 64. neg is 16 pos is 2
i is 64 batch_roi is 18
0 79 rpnclsloss is  0.676485 rpnrgrloss is  0.0584234 roiclsloss is  1.41777 roirgrloss is  0.291101 roi_pos_nums_tmp 16.0
error roi img batch is not 64. neg is 21 pos is 1
i is 94 batch_roi is 22
0 119 rpnclsloss is  0.679048 rpnrgrloss is  0.999414 roiclsloss is  2.48992 roirgrloss is  0.43506 roi_pos_nums_tmp 32.0
error roi img batch is not 64. neg is 54 pos is 4
i is 129 batch_roi is 58
error roi img batch is not 64. neg is 0 pos is 1
i is 139 batch_roi is 1
0 159 rpnclsloss is  0.678318 rpnrgrloss is  0.94671 roiclsloss is  2.24306 roirgrloss is  0.602657 roi_pos_nums_tmp 32.0
error roi img batch is not 64. neg is 44 pos is 4
i is 160 batch_roi is 48
error roi img batch is not 64. neg is 32 pos is 2
i is 166 batch_roi is 34
0 199 rpnclsloss is  0.665428 rpnrgrloss is  0.29245 roiclsloss i

In [17]:
print(rpn_label_temp[np.where(gt_rpn_label==1)[0][:10]])
print(rpn_loc_tmp[np.where(gt_rpn_label==1)[0][:10]])
print(anchor[np.where(gt_rpn_label==1)[0][30:40]])
print(gt_bbox)

[[ 0.5261395   0.47386053]
 [ 0.50610775  0.49389231]
 [ 0.50732124  0.49267885]
 [ 0.50980514  0.49019486]
 [ 0.50961387  0.49038616]
 [ 0.50768661  0.49231347]
 [ 0.5223335   0.47766647]
 [ 0.51177549  0.48822454]
 [ 0.50976241  0.49023756]
 [ 0.50767344  0.49232656]]
[[ 0.          0.          0.1377672   0.03339927]
 [ 0.          0.02019043  0.16665938  0.        ]
 [ 0.          0.06345844  0.03462837  0.        ]
 [ 0.          0.00671508  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.01348028  0.          0.        ]
 [ 0.          0.0293942   0.05539064  0.03232457]
 [ 0.          0.01228669  0.0846479   0.02054499]
 [ 0.          0.06269422  0.03245665  0.03421598]]
[[  48.            0.          560.          512.        ]
 [  64.            0.          576.          512.        ]
 [  80.            0.          592.          512.        ]
 [   0.           16.          432.      

In [13]:
def _fast_rcnn_loc_loss(pred_loc, gt_loc, gt_label, sigma):
    in_weight0 = tf.zeros([tf.shape(gt_loc)[0],tf.shape(gt_loc)[1]],dtype=tf.int32)
    in_weight1 = tf.ones([tf.shape(gt_loc)[0],tf.shape(gt_loc)[1]],dtype=tf.int32)
    in_weight = tf.where((gt_label > 0),in_weight1,in_weight0)
    loc_loss = _modified_smooth_l1(pred_loc, gt_loc, in_weight, sigma)
    loc_loss11 = loc_loss /tf.to_float(tf.shape(tf.where(gt_label>=0))[0])
    return loc_loss,loc_loss11

In [16]:
pred_loc = tf.constant([[0.1,0.1,0.1,0.1],[0.2,0.2,0.2,0.2],[0.3,0.3,0.3,0.3]])
gt_loc   = tf.constant([[0.1,2.1,0.6,0.1],[0.2,2.1,0.6,0.2],[0.3,2.1,0.6,0.3]])
sigma = tf.constant(1.0)
gt_label = tf.constant([1,1,0],dtype=tf.int32)
pre_label = tf.constant([[1,2],[0,3],[2,1.5]],dtype=tf.float32)
loss,loss11 = _fast_rcnn_loc_loss(pred_loc, gt_loc, gt_label, sigma)
mm = tf.to_float(tf.shape(tf.where(gt_label>=0))[0])

In [17]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    loss_tmp,loss11_tmp = sess.run([loss,loss11])
    print("loss",loss_tmp,loss11_tmp)
    print("mm",sess.run(mm))

loss 3.105 1.035
mm 3.0


In [7]:
a = np.arange(24).reshape(2,4,3)
print(a)
a = a[...,::-1]
print(a)

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]]

 [[12 13 14]
  [15 16 17]
  [18 19 20]
  [21 22 23]]]
[[[ 2  1  0]
  [ 5  4  3]
  [ 8  7  6]
  [11 10  9]]

 [[14 13 12]
  [17 16 15]
  [20 19 18]
  [23 22 21]]]
